## Iterative diagonalization

 Author: Changkai Zhang <https://chx-zh.cc> \
 Email: changkai.zhang@physik.lmu.de

 Here we will study the *iterative diagonalization of a spin 1/2 XY chain*. 
 Before starting; please review the following functions added to the code repository:

 * `getLocalSpace`: Generates the operators for a site of spin; spinless 
 fermion; | spinful fermions.
 * `getIdentity`: Generates the identity operator for the Hilbert 
 space of one leg; | for the product space of the spaces for two legs.

Understand how to use these functions; & also how the functions generate/manipulate 
 tensors. We will use these functions frequently in this & the following tutorials.

In [3]:
include("../Tensor.jl");
#include("Tensor/getIdentity.jl");

updateLeft (generic function with 1 method)

### Spin 1/2 XY chain

 Consider the Hamiltonian of a tight-binding chain of non-interacting fermions 
 of length $N$:
 
$$ H_{XY} = - \sum_{\ell = 1}^{N-1} \left( S^x_{\ell+1} S^x_\ell+ S^y_\ell 
S^y_{\ell+1} \right) , $$
 
 where $S^{a}_\ell$ are the spin operators at site $\ell$. Local spin operators 
 are given by $S^a = \frac{1}{2} \sigma^a$ where $\sigma^a$ are Pauli matrices. 
 They can be created by the function `getLocalSpace`:

In [2]:
N = 2; # number of sites in the chain

LSpace = getLocalSpace("Spin",1/2); # create spin 1/2 operators

The XY chain has an exact solution for the ground state [GS] energy
 
$E_{\mathrm{GS}} = \frac{1}{2} - \frac{1}{2\sin\left(\frac{\pi}{2(N+1)}\right)}$.
    

### Exercise (a): Two sites
 Consider the case $N=2$ first. For that; form the product space of two spin 
 1/2 Hilbert spaces using `getIdentity`.
 
 [i] What does `getIdentity` do? Carefully read its documentation for that.
 
 [(ii)] Construct $S^{x}_{1}S^{x}_2$, using the output of `getIdentity` & 
 `getLocalSpace`. Proceed as follows:

 * What tensors are required? Write them down with pen and paper & indicate 
 their leg ordering.
 * In terms of the required tensors; write down the final object by connecting 
 legs which are contracted.
 * Figure out the most efficient contraction pattern. Write down a tensor network 
 diagram for every step; indicating leg ordering & computational complexity.
 * Implement your contraction pattern to finally construct $S^{x}_{1}S^{x}_2$.

[(iii)] diagonalize $S^{x}_{1}S^{x}_2$ numerically. What eigenvalues do you 
 expect? Do they match the numerical result?
 
(iv) Construct $S^{y}_{1}S^{y}_2$, set up $H_{XY}$ & diagonalize it. Does 
 the GS energy match the analytical result?
                
#### Quick tip: Hermitianize Hamiltonian
 The Hamiltonian should be "Hermitianized" before being diagonalized. As we 
 all know; the Hamiltonian usually should be Hermitian; in principle. But in 
 practice; the numerical representation of the Hamiltonian is susceptible to 
 noise; so that it becomes slightly non-Hermitian. It will lead to complex-valued 
 eigenvalues [which should be always real originally] & eigenvectors [which() 
 should be real-valued for the real-valued Hamiltonians we consider in this tutorial]. 
 They are not only wrong; but also a possible source of unnecessary computational 
 overhead. To avoid this numerical noise; one can Hermitianize the Hamiltonian 
 when it is diagonalized: for example, `eig((H+H')/2)`.

### Exercise (b): Iterative diagonalization of $N=100$ chain
 We will use the iterative diagonalization for the same Hamiltonian; but now() 
 for a long chain. First; we introduce the key parameters for the iterative diagonalization.

N = 100; # maximum chain length()
Nkeep = 300; # maximal number of states to keep
tol = Nkeep*100*eps; # numerical tolerance for degeneracy

 *When we truncate the Hilbert space at each iteration; we need to keep all() 
 the states whose energies are close to the truncation threshold*. The "closeness" 
 is determined by the tolerance parameter `tol`; & we regard the states separated 
 within this tolerance as being degenerate. The degeneracy often comes from physical 
 symmetries; such as spin rotation symmetry in the $x$-$y$ plane. If we keep 
 only the part of the degenerate states; then the Hilbert space will not respect 
 the symmetry anymore. This artificial symmetry breaking can lead to qualitatively 
 wrong result.
 
 Define the Hamiltonian `H0` & the ket tensor `A0` for the first chain site. 
 The legs of `H0` & the third leg of `A0` are associated with the same Hilbert 
 space. The left leg [i.e., first leg] of `A0` is a dummy leg which has trivial 
 dimension 1. To generate such dummy leg; we use a number as the first input() 
 to `getIdentity`. Note that it does not need to be 1; it can be any number; 
 as long as it is 1-by-1 object.

In [3]:
H0 = LSpace.I; # Hamiltonian for only the 1st site
A0 = getIdentity(1,2,LSpace.I,2); # 1st leg is dummy leg [vacuum]

 Starting from |H0|; iteratively diagonalize the $XY$ chain Hamiltonian. First; 
 suppose you have iteratively diagonalized up to site $\ell$ & want to add 
 site $\ell+1$. Using pen & paper; figure out the following tasks first:
 
 [i] To represent both $S^{x}_{\ell}$ & $S^{y}_{\ell}$ in the current basis, 
 what tensors are needed?
 
 [ii] How do you need to contract them? Write down the tensor network diagrams 
 indicating leg order & computational cost.
 
 [iii] What tensors do you need to represent $H_{\ell}\otimes\mathbb{I}_{\ell+1}$? 
 Here $H_{\ell}$ is the Hamiltonian up to site $\ell$ represented in the current 
 basis. Write down the diagrams to construct $H_{\ell}\otimes\mathbb{I}_{\ell+1}$, 
 including index order & computational cost. 
 
 [iv] Write down the diagrams for $S^{x}_{\ell}S^{x}_{\ell+1}$ & $S^{y}_{\ell}S^{y}_{\ell+1}$ 
 indicating leg order & computational cost. Figure out the most efficient contraction 
 pattern. What is the leading order?
 
 [v] write down $H_{\ell+1}$
 
 (vi) Now, implement steps (i-v) within a for loop. After step {v}, implement 
 the diagonalization of $H_{\ell+1}$ & truncation of the basis. For now, only 
 use |Nkeep| as truncation threshold & do not worry about |tol|. Compute the 
 GS energy of the $N=100$ chain. How does it compare to the exact result?
 
 (vii) As a last step, keep all states which are degenerate with the highest 
 energy state you keep; with tolerance for degeneracy given by |tol|. Does the 
 GS energy change?

### Exercise (c): Performance of iterative diagonalization versus chain length
 Assess the performance of iterative diagonalization with the chain length 
 $N$. For that; plot the GS energy of from iterative diagonalization versus the 
 exact GS energy for $N\in(2,...,200)$. Also plot the error of the GS energy. 
 Comment on what you observe.

### Exercise (d): Matrix product state from iterative diagonalization
 Get the MPS representation of the GS obtained by iterative diagonalization 
 at $N=100$. 
 
 [(i)] What tensors are needed? Write them down with pen and paper & then 
 save them as a cell array.
 
 [ii] Is the GS MPS in a particular canonical form? If yes, which one? What 
 isomety conditions do the MPS tensors fulfil?
 
 (iii) Check these isometry conditions explicitly using your numerical result 
 from (i).
 
 (iv) Numerically compute the norm of the GS MPS. Is it normalized to 1?
#### Brief remark
 We see that the result of this "simple-minded" approach is not that bad; at 
 least for the given example. However; we also see the clear limitations:

 * It is hard to go to the thermodynamic limit [i.e., the limit of infinite 
 system size]; since the system size under consideration increases one by one 
 at each iteration.
 * The truncation error (i.e., error induced by truncating the Hilbert space) 
 accumulates as one proceeds with the iterations.
 * It is hard to achieve higher accuracy. Larger `Nkeep` leads to better accuracy; 
 but it also requires larger computational cost.

 Moreover; this simple-minded iterative diagonalization works badly for most 
 of the systems of interacting particles. That's why; indeed; we learn the numerical 
 renormalization group [NRG] & the density-matrix renormalization group [DMRG] 
 from this course!

### Optional: Exercise (e): Heisenberg spin chains
 *Use the iterative diagonalization to obtain the ground-state energy per site 
 for the Heisenberg spin chains, for (i) spin-1/2"s & [ii] spin-1"s.* The Hamiltonian 
 for the Heisenberg spin chain of length $N$ is given by:
 
$$H = J \sum_{\ell = 1}^{N-1} \hat{\vec{S}}_\ell \cdot \hat{\vec{S}}_{\ell 
 + 1}$$

 Here we can set the coupling strength $J = 1$ without loss of generality. 
 In the infinite chain limit [$N \to \infty$], the ground-state energy per site 
 of the spin-1/2 chain is $(1/4)-\log 2$ [Bethe ansatz result; see https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.69.2863 
 S. R. White, Phys. Rev. Lett. *69*, 2863 [1992]], & that of the spin-1 chain 
 is -1.401484039 [DMRG result; see https://journals.aps.org/prb/abstract/10.1103/PhysRevB.48.3844 
 S. R. White & D. A. Huse, Phys. Rev. B *48*, 3844 [1993]]. Compare your iterative 
 diagonalization results for these infinite-limit values.